In [54]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm
import matplotlib.pyplot as plt
import time

In [2]:
df_temp = pd.read_csv('회귀_지표면_결과/지면온도최종.txt')
df_temp['날짜'] = pd.to_datetime(df_temp['날짜'], format='%Y%m%d')
df_temp

,Unnamed: 0,날짜,지면온도
0,0,2013-12-21,1.680988
1,1,2014-01-06,4.558961
2,2,2014-01-22,-2.567610
3,3,2014-02-23,10.537078
4,4,2014-03-11,12.563964
5,5,2014-03-27,25.446482
6,6,2014-11-06,18.583097
7,7,2014-12-08,4.070184
8,8,2014-12-24,5.967186
9,9,2015-01-09,1.940758


In [3]:
df_road = pd.read_csv('기상청_노면_습도_온도/0900_지면상태_11to3월_기간.txt')
df_road['날짜'] = pd.to_datetime(df_road['날짜'], format='%Y-%m-%d')
df_road

,Unnamed: 0,지점명,날짜,시간,습도,지면_func
0,0,서울,2013-11-01,09:00,49.0,0
1,1,서울,2013-11-02,09:00,73.0,0
2,2,서울,2013-11-03,09:00,92.0,0
3,3,서울,2013-11-04,09:00,75.0,0
4,4,서울,2013-11-05,09:00,76.0,0
...,...,...,...,...,...,...
447,447,서울,2016-03-27,09:00,68.0,0
448,448,서울,2016-03-28,09:00,53.0,0
449,449,서울,2016-03-29,09:00,45.0,0
450,450,서울,2016-03-30,09:00,71.0,0


In [4]:
regression = df_temp.merge(df_road, left_on='날짜', right_on='날짜')
regression

,Unnamed: 0_x,날짜,지면온도,Unnamed: 0_y,지점명,시간,습도,지면_func
0,0,2013-12-21,1.680988,50,서울,09:00,69.0,0
1,1,2014-01-06,4.558961,66,서울,09:00,59.0,1
2,2,2014-01-22,-2.567610,82,서울,09:00,69.0,0
3,3,2014-02-23,10.537078,114,서울,09:00,59.0,1
4,4,2014-03-11,12.563964,130,서울,09:00,72.0,0
5,5,2014-03-27,25.446482,146,서울,09:00,71.0,0
6,6,2014-11-06,18.583097,156,서울,09:00,88.0,0
7,7,2014-12-08,4.070184,188,서울,09:00,67.0,1
8,8,2014-12-24,5.967186,204,서울,09:00,88.0,0
9,9,2015-01-09,1.940758,220,서울,09:00,64.0,1


In [5]:
regression_result = pd.DataFrame()
regression_result['날짜'] = regression['날짜']
regression_result['지면온도'] = regression['지면온도']
regression_result['노면상태'] = regression['지면_func']
regression_result['습도'] = regression['습도']
regression_result

,날짜,지면온도,노면상태,습도
0,2013-12-21,1.680988,0,69.0
1,2014-01-06,4.558961,1,59.0
2,2014-01-22,-2.567610,0,69.0
3,2014-02-23,10.537078,1,59.0
4,2014-03-11,12.563964,0,72.0
5,2014-03-27,25.446482,0,71.0
6,2014-11-06,18.583097,0,88.0
7,2014-12-08,4.070184,1,67.0
8,2014-12-24,5.967186,0,88.0
9,2015-01-09,1.940758,1,64.0


In [97]:
feature_columns = regression_result.columns.difference(["노면상태"])
x = regression_result[['지면온도']]
y = regression_result['노면상태']

train_x, test_x ,train_y, test_y = train_test_split(x, y, stratify=y,train_size=0.8,test_size=0.2,random_state=8)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(14, 1) (4, 1) (14,) (4,)


In [98]:
model = sm.Logit(train_y, train_x)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.562037
         Iterations 6


In [99]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   노면상태   No. Observations:                   14
Model:                          Logit   Df Residuals:                       13
Method:                           MLE   Df Model:                            0
Date:                Sat, 18 Sep 2021   Pseudo R-squ.:                  0.1377
Time:                        14:44:27   Log-Likelihood:                -7.8685
converged:                       True   LL-Null:                       -9.1246
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
지면온도          -0.1105      0.069     -1.592      0.111      -0.247       0.026
==============================================================================
"""